# 0. Import Dependencies

In [1]:
## data manipulation, statistics and visualization
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis, shapiro, ranksums, ttest_ind
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests
import scipy.signal as signal

## model development and evaluation
import sklearn as sk
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV,GridSearchCV, LeaveOneOut, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import recall_score, precision_score, f1_score, make_scorer, accuracy_score, confusion_matrix
from sklearn.decomposition import PCA


## EEG analysis, DFA, feature selection
import mne
import fathon
from fathon import fathonUtils as fu
import mrmr
from mrmr import mrmr_classif

## system operations
import os
import glob
import openpyxl

# %matplotlib inline
%matplotlib inline

In [2]:
# set the working paths
PATH = r'C:\Users\nikol\home\master_thesis\HRI'
EEG_PATH = os.path.join(PATH,'Data', 'Cleaned EEG signals')
DATA_PATH = os.path.join(PATH,'Data', 'Feature powers.xlsx')
TARGET_PATH = os.path.join(PATH, 'Data', 'Target data.xlsx')

# 1. Load EEG data

In [3]:
robot_indx = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 28, 29, 32, 37, 46, 47, 49] #[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 28, 29, 32, 37, 46, 47, 49]
display_indx = [19, 20, 24, 25, 26, 27, 30, 31, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 48] #[19, 20, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 48]


robot_dict = {}
display_dict = {}
files = glob.glob(EEG_PATH + "\*.set")

for filePath in files:
    fName = os.path.basename(filePath) #filename
    _, ext = os.path.splitext(filePath) #extension (.set)
    p = int(os.path.splitext(fName)[0].split('_')[0]) #split filenames into participant integers
    
    if p in robot_indx: # add each file with index in participant indeces to robot group
        robot_dict[p] = mne.io.read_raw_eeglab(filePath, preload=False)
    elif p in display_indx: # and display group
        display_dict[p] = mne.io.read_raw_eeglab(filePath, preload=False)

Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\10_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\11_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\12_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\13_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\14_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\15_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\16_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\17_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\18_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\19_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\20_final.fdt
Reading C:\Users\nikol\home\master_thesis\HRI\Data\Cleaned EEG signals\21_fi

In [4]:
# check participant count per condition
n_robot = len(robot_dict)
n_display = len(display_dict)

print("robot-group dict:", n_robot, 'participant(s)')
print("display-group dict:", n_display, 'participant(s)')

# define dictionary to map 10-20 channel names to eeg sensor locations
channel_dict = {}
channels = ['E5', 'E10', 'E12', 'E20', 'E24', 'E28', 'E35', 'E39', 'E42', 'E50', 'E52', 'E60', 'E6', 'E18', 'E58', 'E34', 'E37','Cz']
channel_names = ['Fp2', 'Fp1', 'F3', 'C3', 'T7', 'P3', 'O1', 'O2', 'P4', 'C4', 'T8', 'F4', 'Fz', 'F7', 'F8', 'Pz', 'Oz', 'Cz']

x = 0
for ch in channels:
    channel_dict[ch] = channel_names[x]
    x+=1

# change channel names to 10-20 system
for participant in robot_dict:
    robot_dict[participant].pick_channels(channels)
    robot_dict[participant].rename_channels(channel_dict)
    
for participant in display_dict:
    display_dict[participant].pick_channels(channels)
    display_dict[participant].rename_channels(channel_dict)

robot-group dict: 20 participant(s)
display-group dict: 21 participant(s)


In [5]:
children_indx = list(set(robot_indx + display_indx))

# 2. Extract DFA exponents from EEG

generate a dictionary with EEG recordings of the two groups

In [6]:
#combine the the dictionaries with EEG recordings (the robot group EEG and the display group EEG)
eeg_dict = {**robot_dict, **display_dict}

In [7]:
#set frequency and period for extracting EEG data
sampling_freq = eeg_dict[9].info['sfreq']
start_stop_seconds = np.array([0, 420])
start_sample, stop_sample = (start_stop_seconds * sampling_freq).astype(int)

In [8]:
# print all of the 18 channels
print(eeg_dict[9].ch_names, end = '')

['Fp2', 'Fz', 'Fp1', 'F3', 'F7', 'C3', 'T7', 'P3', 'Pz', 'O1', 'Oz', 'O2', 'P4', 'C4', 'T8', 'F8', 'F4', 'Cz']

In [9]:
# choose the index of the EEG channel
# change that index to calculate DFA for each channel
channel_index = 0

In [10]:
# create the dictionary that will contain the EEG data for all of the children and the specific channel
eeg_data = {}


for child in children_indx:
      eeg_data[child] = eeg_dict[child][channel_index, start_sample:stop_sample][0]

extract the DFA exponents and save them in text files

In [11]:
#define the epochs to divide the signal
winSizes = fu.linRangeByStep(10, 420)
revSeg = True
polOrd = 3

In [13]:
%%time
np.random.seed(42)
# create a list with DFA exponents for all of the children and the specific channel

dfa_list = []

for child in children_indx:
    eeg_data[child] = fu.toAggregated(eeg_data[child])
    pydfa = fathon.DFA(eeg_data[child])
    n, F = pydfa.computeFlucVec(winSizes, revSeg=revSeg, polOrd=polOrd)
    H, H_intercept = pydfa.fitFlucVec()
    
    dfa_list.append(H)

Wall time: 3min 36s


In [37]:
# save the DFA exponents in a text file (for each children and for the specific channel)
with open(f'./Data/{eeg_dict[9].ch_names[channel_index]}_dfa.txt', 'w') as file:
    for exp in dfa_list:
        file.write(f'{exp}\n')

Then change the channel index and repeat the DFA calculations to obtain DFA exponents, for every children, for each of the 18 channels

# 3. Prepare the spectral and spectral + dfa dataframes

get the subjects to remove from the data

### spectral dataframe

In [ ]:
PATH = r'C:\Users\nikol\home\master_thesis\HRI'
EEG_PATH = os.path.join(PATH,'Data', 'Cleaned EEG signals')
DATA_PATH = os.path.join(PATH,'Data', 'Feature powers.xlsx')
TARGET_PATH = os.path.join(PATH, 'Data', 'Target data.xlsx')

In [ ]:
DATA_PATH

In [ ]:
df = pd.read_csv(r'C:\Users\nikol\home\master_thesis\HRI\Data\Feature powers.xlsx')

In [ ]:
df.head()

In [ ]:
#remove the pilot subjects

df = df.drop(list(range(0,8)), axis=0)
df.reset_index(inplace = True)
df.drop(['index'], inplace = True, axis=1)

In [ ]:
# rename the dataframe
X_spec = df

In [ ]:
# fix the indexes
X_spec.reset_index(inplace = True)
X_spec.drop(['index'], inplace = True, axis=1)

In [ ]:
X_spec.head()

### prepare dfa data

In [25]:
# create a dictionary that will contain the DFA exponents
dfa_data = dict((ch, []) for ch in robot_dict[9].ch_names)

In [35]:
# read each of the DFA text files
for ch in robot_dict[9].ch_names:

    data =  open(f'./Data/DFA_each_channel/{ch}_dfa.txt', 'r')

    dataString = data.read()
    text_dfa = dataString.split('\n')
                 
    dfa_data[ch].append(text_dfa)

    

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [28]:
for ch in robot_dict[9].ch_names:
    dfa_data[ch] = dfa_data[ch][0]

In [29]:
dfa_data = pd.DataFrame(dfa_data)

In [30]:
data.close()

In [ ]:
#transform the DFA features into floats
for ch in robot_dict[9].ch_names:
    dfa_data[ch] = dfa_data[ch].astype(float)

In [ ]:
dfa_data.info()

### remove the not-tested subjects

In [ ]:
# get the indexes of subjects to remove
remove_list = [9, 18, 45, 46, 48, 42, 43, 44, 47]
remove_index = []


for num in remove_list:
    remove_index.append(children_indx.index(num))

print(remove_index)

remove not-tested subjects from spectral + dfa data

In [ ]:
dfa_data.drop(remove_index, axis=0, inplace=True)

In [ ]:
# fix the indexes
dfa_data.reset_index(inplace = True)
dfa_data.drop(['index'], inplace = True, axis=1)

remove not-tested subjects from spectral data

In [ ]:
X_spec.drop(remove_index, axis=0, inplace=True)

In [ ]:
# fix the indexes
X_spec.reset_index(inplace = True)
X_spec.drop(['index'], inplace = True, axis=1)

### prepare target data

In [ ]:
target_data = pd.read_csv(r'C:\Users\nikol\home\master_thesis\HRI\Processed data from Stephanie\Target data.csv')

In [ ]:
average_score = np.mean(target_data['TOTAL CORRECT'])

def classification_transf(row):
    if row['TOTAL CORRECT'] > average_score:
        return 1
    elif row['TOTAL CORRECT'] < average_score:
        return 0

target_data['Successfull'] = target_data.apply(classification_transf, axis=1)

y = target_data['Successfull']

In [ ]:
y = y.values.reshape(-1,1)

# 4. EDA

In [ ]:
spec_viz = df.join(pd.Series(np.squeeze(y)).rename('Success'))
dfa_viz = dfa_data.join(pd.Series(np.squeeze(y)).rename('Success'))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,5))
fig.tight_layout(pad=1, w_pad=2, h_pad=1.0)


dfa_viz[dfa_viz['Success']== 0].iloc[:,:-1].boxplot(ax=axs.ravel()[0])
dfa_viz[dfa_viz['Success']== 1].iloc[:,:-1].boxplot(ax=axs.ravel()[1])

axs.ravel()[0].set_title('Bad learners')
axs.ravel()[1].set_title('Good learners')

In [ ]:

np.random.seed(42)
fig, axs = plt.subplots(6, 3, figsize=(20,15))
fig.tight_layout(pad=1, w_pad=2, h_pad=1.0)
dfa_ch = robot_dict[9].ch_names

for ch, ax in zip(dfa_ch, axs.ravel()):
    ax.boxplot([dfa_viz[dfa_viz['Success']== 0].iloc[:,:-1][f'{ch}'],dfa_viz[dfa_viz['Success']== 1].iloc[:,:-1][f'{ch}']])
    ax.set_xticklabels(['Bad learners', 'Good learners'])
    ax.set_title(f'{ch}')
    ax.grid(visible=True)

# 5. Statistical tests

#### spectral

In [ ]:
ttest_ind(spec_viz[spec_viz['Success']==0]['Alpha_Fp2'], spec_viz[spec_viz['Success']==1]['Alpha_Fp2'], alternative='less').pvalue

In [ ]:
#Shapiro-Wilk test 
for spec in ['Alpha', 'Beta', 'Theta']:
    for ch in robot_dict[9].ch_names:
        print(f'{spec}_{ch}:', shapiro(spec_viz[f'{spec}_{ch}']))

parametric test

In [ ]:
spec_pvalues = []
for spec in ['Alpha', 'Beta', 'Theta']:
    for ch in robot_dict[9].ch_names:
    
        print(f'{spec}_{ch} feature: ',ttest_ind(spec_viz[spec_viz['Success']==0][f'{spec}' + '_' + f'{ch}'], spec_viz[spec_viz['Success']==1][f'{spec}' + '_' + f'{ch}'], alternative='less').pvalue)
        spec_pvalues.append(ttest_ind(spec_viz[spec_viz['Success']==0][f'{spec}' + '_' + f'{ch}'], spec_viz[spec_viz['Success']==1][f'{spec}' + '_' + f'{ch}'], alternative='less').pvalue)

In [ ]:
alpha = 0.05

rejected_spec, p_adj_spec, _, alpha_corrected_spec = multipletests(spec_pvalues, 
                                                              alpha=alpha,
                                                              method='bonferroni', 
                                                              is_sorted=False, 
                                                              returnsorted=False)

print(np.sum(rejected_spec))

print(alpha_corrected_spec)

non-parametric test

In [ ]:
spec_pvalues = []
for spec in ['Alpha', 'Beta', 'Theta']:
    for ch in robot_dict[9].ch_names:
    
        print(f'{spec}_{ch} feature: ',ranksums(spec_viz[spec_viz['Success']==0][f'{spec}' + '_' + f'{ch}'], spec_viz[spec_viz['Success']==1][f'{spec}' + '_' + f'{ch}'], alternative='greater').pvalue< 0.05)
        spec_pvalues.append(ranksums(spec_viz[spec_viz['Success']==0][f'{spec}' + '_' + f'{ch}'], spec_viz[spec_viz['Success']==1][f'{spec}' + '_' + f'{ch}'], alternative='greater').pvalue)




In [ ]:
alpha = 0.05

rejected_spec, p_adj_spec, _, alpha_corrected_spec = multipletests(spec_pvalues, 
                                                              alpha=alpha,
                                                              method='bonferroni', 
                                                              is_sorted=False, 
                                                              returnsorted=False)

print(np.sum(rejected_spec))

print(alpha_corrected_spec)

#### dfa + spectral

In [ ]:
# test for normality

# skewness (-2, +2) and kurtosis(-7, +7)

print('Skewness in {ch}: ', skew(dfa_viz.Fp2))
print('Kurtosis:', kurtosis(dfa_viz.Fp2))



#Shapiro-Wilk test 
for ch in robot_dict[9].ch_names:
    print(f'{ch}:', shapiro(dfa_viz[f'{ch}']))

In [ ]:
# T-test for the means of two independent samples
dfa_pvalues = []

for ch in robot_dict[9].ch_names:
    
    print(f'{ch} channel: ',ttest_ind(dfa_viz[dfa_viz['Success']==0][f'{ch}'], dfa_viz[dfa_viz['Success']==1][f'{ch}'], alternative='less'))
    dfa_pvalues.append(ttest_ind(dfa_viz[dfa_viz['Success']==0][f'{ch}'], dfa_viz[dfa_viz['Success']==1][f'{ch}'], alternative='less').pvalue)

In [ ]:
rejected_dfa, p_adj_dfa, _, alpha_corrected_dfa = multipletests(dfa_pvalues, 
                                                              alpha=alpha,
                                                              method='bonferroni', 
                                                              is_sorted=False, 
                                                              returnsorted=False)

print(np.sum(rejected_dfa))

print(alpha_corrected_dfa)

# 6. Feature Selection

#### minimum Redundancy - Maximum Relevance

spectral data

In [ ]:
selected_features_spec = mrmr_classif(X=X_spec, y=y, K=5)

In [ ]:
selected_features_spec

In [ ]:
X_spec = X_spec[selected_features_spec]

In [ ]:
X_spec.shape

dfa data

In [ ]:
selected_features_dfa = mrmr_classif(X=dfa_data, y=y, K=3)

In [ ]:
selected_features_dfa

In [ ]:
dfa_data = dfa_data[selected_features_dfa]

In [ ]:
dfa_data.shape

# 7. Develop the models

In [ ]:
random.seed(42)

In [ ]:
param_grid_svc = { 'C':[10.0, 100.0,1000.0, 10000.0, 100000.0],
              'kernel':['rbf','poly'],
              'degree':[1,2,3,4],
              'gamma': [10.0, 100.0,300]}


leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

param_grid_knn = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)


In [ ]:
# normalize the data
scaler = MinMaxScaler()
scaled_features_spec = scaler.fit_transform(X_spec.values)
X_spec_norm = pd.DataFrame(scaled_features_spec, index=X_spec.index, columns=X_spec.columns)

In [ ]:
scaled_features_dfa = scaler.fit_transform(X_dfa.values)
X_dfa_norm = pd.DataFrame(scaled_features_dfa, index=X_dfa.index, columns=X_dfa.columns)

## spectral data

#### SVM

In [ ]:
svc_spec = GridSearchCV(SVC(random_state=42),
                                 param_grid_svc,
                                 cv=LeaveOneOut(), 
                                 n_jobs=-1)
svc_spec.fit(X_spec_norm, y)

In [ ]:
svc_spec.best_params_

#### KNN

In [ ]:
knn_spec = GridSearchCV(KNeighborsClassifier(),
                                 param_grid_knn,
                                 cv=LeaveOneOut(), 
                                 n_jobs=-1)
knn_spec.fit(X_spec, y)

In [ ]:
knn_spec.best_params_

## spectral + DFA

In [ ]:
X_dfa = df.join(dfa_data)

#### SVM

In [ ]:
svc_dfa = GridSearchCV(SVC(random_state=42),
                                 param_grid_svc,
                                 cv=LeaveOneOut(), 
                                 n_jobs=-1)
svc_dfa.fit(X_dfa, y)

In [ ]:
svc_dfa.best_params_

#### KNN

In [ ]:
knn_dfa = GridSearchCV(KNeighborsClassifier(),
                                 param_grid_knn,
                                 cv=LeaveOneOut(), 
                                 n_jobs=-1)
knn_dfa.fit(X_dfa_norm, y)

In [ ]:
knn_dfa.best_params_

# 8. Evaluate the models

In [ ]:
# define a function that compares the LOO perfromance of a set of predetrmined models 
def cv_comparison(models, X, y):
    # Initiate a DataFrame for the averages and a list for all measures
    loo = LeaveOneOut()
    cv_metrics = pd.DataFrame()
    y_true = []
    y_pred = []
    random.seed(42)

    for model in models:
        for i, j in loo.split(X):
            X_train, X_test = X.iloc[i,:], X.iloc[j, :]
            y_train, y_test = y[i], y[j]
            model.fit(X_train, y_train)
            y_hat = model.predict(X_test)
            y_true.append(y_test[0])
            y_pred.append(y_hat[0])
            
            accuracy = accuracy_score(y_true, y_pred)
            recall = recall_score(y_true, y_pred)
            precision = precision_score(y_true, y_pred)
            f1 = f1_score(y_true, y_pred)

            cv_metrics[str(model)] = [recall, precision, f1, accuracy]
    cv_metrics.index = ['Recall', 'Precision', 'F1-score', 'Accuracy']
    return cv_metrics, recall, precision, f1, accuracy

## spectral

In [ ]:
print(f'SVM hyperparameters: {svc_spec.best_params_}')
print(f'KNN hyperparameters: {knn_spec.best_params_}')

In [ ]:
svm_spec_model = SVC(C=100,
                    degree=1,
                    gamma=300,
                    kernel='rbf', random_state=42)

knn_spec_model = KNeighborsClassifier(leaf_size=1,
                                     n_neighbors = 9,
                                     p=2)

zeroR_spec = DummyClassifier(strategy='most_frequent')

models = [svm_spec_model, knn_spec_model, zeroR]

In [ ]:
comp_spec, recall, precision, f1, accuracy = cv_comparison(models, X_spec, y)

In [ ]:
comp_spec

### DFA + Spectral

In [ ]:
print(f'SVM hyperparameters: {svc_dfa.best_params_}')
print(f'KNN hyperparameters: {knn_dfa.best_params_}')

In [ ]:
svm_dfa_model = SVC(C=10,
                    degree=1,
                    gamma=10,
                    kernel='poly')

knn_dfa_model = KNeighborsClassifier(leaf_size=1,
                                     n_neighbors = 7,
                                     p=2)

zeroR_dfa = DummyClassifier(strategy='most_frequent')

models = [svm_dfa_model, knn_dfa_model, zeroR_dfa]

In [ ]:
comp_dfa, recall, precision, f1, accuracy = cv_comparison(models, X_dfa, y)

In [ ]:
comp_dfa